<a href="https://colab.research.google.com/github/YousefAbua/Intro-To-DL/blob/main/HW5/HW5_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader

In [2]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [3]:
french_to_english = [

      ("J'ai froid", "I am cold"),

      ("Tu es fatigué", "You are tired"),

      ("Il a faim", "He is hungry"),

      ("Elle est heureuse", "She is happy"),

      ("Nous sommes amis", "We are friends"),

      ("Ils sont étudiants", "They are students"),

      ("Le chat dort", "The cat is sleeping"),

      ("Le soleil brille", "The sun is shining"),

      ("Nous aimons la musique", "We love music"),

      ("Elle parle français couramment", "She speaks French fluently"),

      ("Il aime lire des livres", "He enjoys reading books"),

      ("Ils jouent au football chaque week-end", "They play soccer every weekend"),

      ("Le film commence à 19 heures", "The movie starts at 7 PM"),

      ("Elle porte une robe rouge", "She wears a red dress"),

      ("Nous cuisinons le dîner ensemble", "We cook dinner together"),

      ("Il conduit une voiture bleue", "He drives a blue car"),

      ("Ils visitent souvent des musées", "They visit museums often"),

      ("Le restaurant sert une délicieuse cuisine", "The restaurant serves delicious food"),

      ("Elle étudie les mathématiques à l'université", "She studies mathematics at university"),

      ("Nous regardons des films le vendredi", "We watch movies on Fridays"),

      ("Il écoute de la musique en faisant du jogging", "He listens to music while jogging"),

      ("Ils voyagent autour du monde", "They travel around the world"),

      ("Le livre est sur la table", "The book is on the table"),

      ("Elle danse avec grâce", "She dances gracefully"),

      ("Nous célébrons les anniversaires avec un gâteau", "We celebrate birthdays with cake"),

      ("Il travaille dur tous les jours", "He works hard every day"),

      ("Ils parlent différentes langues", "They speak different languages"),

      ("Les fleurs fleurissent au printemps", "The flowers bloom in spring"),

      ("Elle écrit de la poésie pendant son temps libre", "She writes poetry in her free time"),

      ("Nous apprenons quelque chose de nouveau chaque jour", "We learn something new every day"),

      ("Le chien aboie bruyamment", "The dog barks loudly"),

      ("Il chante magnifiquement", "He sings beautifully"),

      ("Ils nagent dans la piscine", "They swim in the pool"),

      ("Les oiseaux gazouillent le matin", "The birds chirp in the morning"),

      ("Elle enseigne l'anglais à l'école", "She teaches English at school"),

      ("Nous prenons le petit déjeuner ensemble", "We eat breakfast together"),

      ("Il peint des paysages", "He paints landscapes"),

      ("Ils rient de la blague", "They laugh at the joke"),

      ("L'horloge tic-tac bruyamment", "The clock ticks loudly"),

      ("Elle court dans le parc", "She runs in the park"),

      ("Nous voyageons en train", "We travel by train"),

      ("Il écrit une lettre", "He writes a letter"),

      ("Ils lisent des livres à la bibliothèque", "They read books at the library"),

      ("Le bébé pleure", "The baby cries"),

      ("Elle étudie dur pour les examens", "She studies hard for exams"),

      ("Nous plantons des fleurs dans le jardin", "We plant flowers in the garden"),

      ("Il répare la voiture", "He fixes the car"),

      ("Ils boivent du café le matin", "They drink coffee in the morning"),

      ("Le soleil se couche le soir", "The sun sets in the evening"),

      ("Elle danse à la fête", "She dances at the party"),

      ("Nous jouons de la musique au concert", "We play music at the concert"),

      ("Il cuisine le dîner pour sa famille", "He cooks dinner for his family"),

      ("Ils étudient la grammaire française", "They study French grammar"),

      ("La pluie tombe doucement", "The rain falls gently"),

      ("Elle chante une chanson", "She sings a song"),

      ("Nous regardons un film ensemble", "We watch a movie together"),

      ("Il dort profondément", "He sleeps deeply"),

      ("Ils voyagent à Paris", "They travel to Paris"),

      ("Les enfants jouent dans le parc", "The children play in the park"),

      ("Elle se promène le long de la plage", "She walks along the beach"),

      ("Nous parlons au téléphone", "We talk on the phone"),

      ("Il attend le bus", "He waits for the bus"),

      ("Ils visitent la tour Eiffel", "They visit the Eiffel Tower"),

      ("Les étoiles scintillent la nuit", "The stars twinkle at night"),

      ("Elle rêve de voler", "She dreams of flying"),

      ("Nous travaillons au bureau", "We work in the office"),

      ("Il étudie l'histoire", "He studies history"),

      ("Ils écoutent la radio", "They listen to the radio"),

      ("Le vent souffle doucement", "The wind blows gently"),

      ("Elle nage dans l'océan", "She swims in the ocean"),

      ("Nous dansons au mariage", "We dance at the wedding"),

      ("Il gravit la montagne", "He climbs the mountain"),

      ("Ils font de la randonnée dans la forêt", "They hike in the forest"),

      ("Le chat miaule bruyamment", "The cat meows loudly"),

      ("Elle peint un tableau", "She paints a picture"),

      ("Nous construisons un château de sable", "We build a sandcastle"),

      ("Il chante dans le chœur", "He sings in the choir")

  ]


In [4]:
class TranslationDataset(Dataset):
    def __init__(self, dataset, word_to_index):
        self.dataset = dataset
        self.word_to_index = word_to_index

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        input_sentence, target_sentence = self.dataset[idx]
        input_indices = [self.word_to_index[word] for word in input_sentence.split()] + [EOS_token]
        target_indices = [self.word_to_index[word] for word in target_sentence.split()] + [EOS_token]
        return torch.tensor(input_indices, dtype=torch.long), torch.tensor(target_indices, dtype=torch.long)

In [7]:
SOS_token = 0
EOS_token = 1

word_to_index = {"SOS": SOS_token, "EOS": EOS_token}
for pair in french_to_english:
    for word in pair[0].split() + pair[1].split():
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)

index_to_word = {i: word for word, i in word_to_index.items()}

# Creating a DataLoader to batch and shuffle the dataset
translation_dataset = TranslationDataset(french_to_english, word_to_index)
dataloader = DataLoader(translation_dataset, batch_size=1, shuffle=True)

In [8]:
class Transformer(nn.Module):
    def __init__(self, input_vocab_size, target_vocab_size, hidden_size, num_layers=2, num_heads=8, dropout=0.1):
        super(Transformer, self).__init__()
        self.embedding_input = nn.Embedding(input_vocab_size, hidden_size)
        self.embedding_target = nn.Embedding(target_vocab_size, hidden_size)
        self.transformer = nn.Transformer(
            d_model=hidden_size,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=hidden_size * 4,
            dropout=dropout
        )
        self.fc = nn.Linear(hidden_size, target_vocab_size)

    def forward(self, input, target):
        input_embedded = self.embedding_input(input)
        target_embedded = self.embedding_target(target)

        input_pad_mask = self.generate_padding_mask(input)
        target_pad_mask = self.generate_padding_mask(target)
        target_subsequent_mask = self.generate_subsequent_mask(target)

        input_embedded = input_embedded.permute(1, 0, 2)
        target_embedded = target_embedded.permute(1, 0, 2)

        output = self.transformer(
            input_embedded,
            target_embedded,
            src_key_padding_mask=input_pad_mask,
            tgt_key_padding_mask=target_pad_mask,
            memory_key_padding_mask=input_pad_mask,
            tgt_mask=target_subsequent_mask
        )

        output = self.fc(output)
        return output.permute(1, 0, 2)

    def generate_padding_mask(self, sequence):
        mask = (sequence == word_to_index["EOS"])
        return mask

    def generate_subsequent_mask(self, sequence):
        mask = (torch.triu(torch.ones(sequence.size(1), sequence.size(1))) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [9]:
def train(model, dataloader, criterion, optimizer, n_epochs):
    for epoch in range(n_epochs):
        total_loss = 0
        total_correct = 0
        total_examples = 0

        model.train()  # Set the model to training mode

        for input_tensor, target_tensor in dataloader:
            input_tensor = input_tensor.to(device)
            target_tensor = target_tensor.to(device)

            optimizer.zero_grad()

            output = model(input_tensor, target_tensor[:, :-1])  # Exclude EOS token from target
            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            target_tensor = target_tensor[:, 1:].contiguous().view(-1)  # Exclude SOS token from target

            loss = criterion(output, target_tensor)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(output, 1)
            correct = (predicted == target_tensor).sum().item()
            total_correct += correct
            total_examples += target_tensor.size(0)  # Use target tensor size for total examples

        avg_loss = total_loss / len(dataloader)
        training_accuracy = total_correct / total_examples  # Calculate training accuracy

        print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {avg_loss:.4f}, Training Accuracy: {training_accuracy:.4f}")


In [10]:
# Assuming all words in the dataset + 'SOS' and 'EOS' tokens are included in word_to_index
input_size = len(word_to_index)
hidden_size = 128
output_size = len(word_to_index)

model = Transformer(input_size, output_size, hidden_size).to(device)

# Set the learning rate for optimization
learning_rate = 0.0005

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Set number of epochs for training
epochs = 10

train(model, dataloader, criterion, optimizer, epochs)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5109: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch [1/10], Loss: 5.1369, Training Accuracy: 0.2114
Epoch [2/10], Loss: 4.1192, Training Accuracy: 0.2429
Epoch [3/10], Loss: 3.3270, Training Accuracy: 0.3343
Epoch [4/10], Loss: 2.6564, Training Accuracy: 0.4257
Epoch [5/10], Loss: 2.1015, Training Accuracy: 0.5429
Epoch [6/10], Loss: 1.6234, Training Accuracy: 0.7114
Epoch [7/10], Loss: 1.2096, Training Accuracy: 0.8600
Epoch [8/10], Loss: 0.8947, Training Accuracy: 0.9200
Epoch [9/10], Loss: 0.6372, Training Accuracy: 0.9686
Epoch [10/10], Loss: 0.4554, Training Accuracy: 0.9886
